In [2]:
import numpy as np
import pandas as pd
from scipy.io.idl import readsav
import matplotlib as mpl
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.constants import astropyconst20 as const
from astropy.time import Time

/var/folders/vm/nfh3cfj106121z2ph22yffx40000gq/T/ipykernel_41416/991110439.py:2: DeprecationWarning: Please use `readsav` from the `scipy.io` namespace, the `scipy.io.idl` namespace is deprecated.
  from scipy.io.idl import readsav


# Integrating away the MinXSS data denominators

We'll define some functions to handle integrating across the MinXSS science data, which are in photons / s / cm2 / keV. These first two functions will remove the /keV and /s, respectively. Then we create a function to account for the fact that we took these measurements at 1 AU, which gets rid of the /cm2.

In [3]:
def integrate_spectrum_energy(spectral_irradiance, energy):
    #energy_threshold = 1.5  # [keV]; below this, data is noise-dominated
    #spectral_irradiance = spectral_irradiance[:, energy > energy_threshold]
    #energy = energy[energy > energy_threshold]
    irradiance_masked = np.ma.array(spectral_irradiance, mask=np.isnan(spectral_irradiance))  # Causes the units to get dropped
    return np.trapz(irradiance_masked, energy)


def integrate_spectrum_time(irradiance, time_jd):
    time_seconds = (time_jd - time_jd[0]) * 86400 * u.second
    return np.trapz(irradiance, time_seconds)


def integrate_photon_flux_1au(fluxes):
    try:
        result = [flux * 4 * np.pi * ((1*u.AU).to(u.cm))**2 for flux in fluxes]
    except TypeError:
        result = fluxes * 4 * np.pi * ((1*u.AU).to(u.cm))**2
    return result

# Convenience functions

In [4]:
def extract_time_jd(minxsslevel1):
    return np.array([minxsslevel1['time'][i]['jd'][0] for i in range(len(minxsslevel1))])

def minxss_near_flare_peak(minxss_time_jd, goes_peak_time_jd, tolerance_pm_minutes=5):
    tolerance_fraction_of_day = tolerance_pm_minutes / 1440
    array = np.asarray(minxss_time_jd)
    idx = (np.abs(minxss_time_jd - goes_peak_time_jd)).argmin()
    if np.abs(minxss_time_jd[idx] - goes_peak_time_jd) <= tolerance_fraction_of_day:
        return idx
    else:
        return np.nan

def convert_photons_to_energy(energy, photons):
    energy_erg = u.keV.to(u.erg, energy) * (u.erg/u.photon)
    try:
        result = [photon * energy_erg for photon in photons]
    except TypeError:
        result = photons * energy_erg
    return result


def convert_kev_to_angstrom(kev):
    wave = const.h.to('keV s') * const.c / kev
    wave = wave.to('angstrom')
    return wave

# Read data

In [5]:
%%time
data_path = '/Users/masonjp2/Dropbox/minxss_dropbox/data/fm1/level1/'
data = readsav('{}minxss1_l1_mission_length_v4.0.0.sav'.format(data_path))
minxsslevel1 = data.minxsslevel1.x123[0].copy()
spectral_irradiance = np.stack(minxsslevel1['irradiance']) * (u.photon / u.second / u.centimeter**2 / u.keV)
time_jd = extract_time_jd(minxsslevel1)
energy = minxsslevel1[0]['energy']
wavelengths = convert_kev_to_angstrom(energy * u.keV)
goes_long_wave_indices = np.where((wavelengths.value >= 1) & (wavelengths.value <= 8))[0]
energy_long = energy[goes_long_wave_indices]

goes_events_path = '/Users/masonjp2/Dropbox/Research/Data/GOES/events/'
data_goes = readsav('{}GOES_events_MinXSS1_era.sav'.format(goes_events_path))
goes_events = data_goes.goesevents.copy()
goes_start_jd = goes_events['eventstarttimejd']
goes_peak_jd = goes_events['eventpeaktimejd']
goes_end_jd = goes_start_jd + 1/24  # Nearly all flares are << 1 hour. The post-flare time intensity is much smaller and will add little to the time-integrated value

CPU times: user 25.8 s, sys: 3.09 s, total: 28.9 s
Wall time: 29.9 s


# Energy for 3 select flares, corresponding to CPHLARE events (subset of GOES events) -- spot checking

In [13]:
def flare_total_energy(flare_spectral_irradiance, preflare_spectral_irradiance, flare_mask, preflare_mask): 
    flare_spectral_irradiance_erg = convert_photons_to_energy(energy_long, flare_spectral_irradiance[:, goes_long_wave_indices])
    preflare_spectral_irradiance_erg = convert_photons_to_energy(energy_long, preflare_spectral_irradiance[:, goes_long_wave_indices])
    irradiance = integrate_spectrum_energy(flare_spectral_irradiance_erg, energy_long) * (u.erg / u.second / u.centimeter**2)
    preflare_irradiances = integrate_spectrum_energy(preflare_spectral_irradiance_erg, energy_long) * (u.erg / u.second / u.centimeter**2)
    preflare_irradiance = np.median(preflare_irradiances)
    flare_irradiance = (irradiance.value - preflare_irradiance.value) * irradiance.unit
    erg_flux = integrate_spectrum_time(flare_irradiance, time_jd[flare_mask])
    total_energy = integrate_photon_flux_1au(erg_flux)
    return total_energy

In [14]:


# Grab data for 3 MinXSS flares also observed by CPHLARE class work looking at GOES flares
flare1_peak_time = '2016-07-24 06:20:00Z'
flare2_peak_time = '2016-11-29 07:10:00Z'
flare3_peak_time = '2017-02-22 13:27:00Z'
flare1_mask = (time_jd >= Time('2016-07-24 05:50:00Z').jd) & (time_jd <= Time('2016-07-24 07:15:00Z').jd)
flare2_mask = (time_jd >= Time('2016-11-29 07:05:00Z').jd) & (time_jd <= Time('2016-11-29 07:29:00Z').jd)
flare3_mask = (time_jd >= Time('2017-02-22 12:59:00Z').jd) & (time_jd <= Time('2017-02-22 14:26:00Z').jd)
preflare1_mask = (time_jd >= Time('2016-07-24 05:45:00Z').jd) & (time_jd <= Time('2016-07-24 05:59:00Z').jd)
preflare2_mask = (time_jd >= Time('2016-11-29 06:00:00Z').jd) & (time_jd <= Time('2016-11-29 07:05:00Z').jd)
preflare3_mask = (time_jd >= Time('2017-02-22 12:30:00Z').jd) & (time_jd <= Time('2017-02-22 12:59:00Z').jd)

flare1_spectral_irradiance = spectral_irradiance[flare1_mask, :]
flare2_spectral_irradiance = spectral_irradiance[flare2_mask, :]
flare3_spectral_irradiance = spectral_irradiance[flare3_mask, :]
preflare1_spectral_irradiance = spectral_irradiance[preflare1_mask, :]
preflare2_spectral_irradiance = spectral_irradiance[preflare2_mask, :]
preflare3_spectral_irradiance = spectral_irradiance[preflare3_mask, :]

print(flare_total_energy(flare1_spectral_irradiance, preflare1_spectral_irradiance, flare1_mask, preflare1_mask))
print(flare_total_energy(flare2_spectral_irradiance, preflare2_spectral_irradiance, flare2_mask, preflare2_mask))
print(flare_total_energy(flare3_spectral_irradiance, preflare3_spectral_irradiance, flare3_mask, preflare3_mask))

6.312590996753241e+28 erg
7.807249092757888e+27 erg
1.6325161860519202e+28 erg
